In [1]:
import sys
sys.path.append("/persist/maplab_ws/src/ninebot_gio_coupling")
from gnsspylib.gps_time import GPSTime, gpst_to_utc, utc_to_gpst, adjweek, gen_gpstime_from_bds
from gnsspylib.sat_no_prn_id import *
from gnsspylib.gen_nav_msg import gen_nav_bag_msg
from gnsspylib.gen_obs_msg import gen_bag_msg_list
from ninebot_gio_coupling.msg import StampedFloat64Array

In [2]:
import georinex as gr
obs_dat = gr.load("/persist/catkin_ws/src/GraphGNSSLib/global_fusion/dataset/gps_solution_TST2/2020_06_03_TST_03.obs")
base_dat_c = gr.load("/persist/catkin_ws/src/GraphGNSSLib/global_fusion/dataset/gps_solution_TST2/hksc155c.20o")
nav_dat_n_c = gr.load("/persist/catkin_ws/src/GraphGNSSLib/global_fusion/dataset/gps_solution_TST2/hksc155c.20n")
nav_dat_b_c = gr.load("/persist/catkin_ws/src/GraphGNSSLib/global_fusion/dataset/gps_solution_TST2/hksc155c.20b")
nav_dat_l_c = gr.load("/persist/catkin_ws/src/GraphGNSSLib/global_fusion/dataset/gps_solution_TST2/hksc155c.20l")
nav_dat_g_c = gr.load("/persist/catkin_ws/src/GraphGNSSLib/global_fusion/dataset/gps_solution_TST2/hksc155c.20g")
base_dat_d = gr.load("/persist/catkin_ws/src/GraphGNSSLib/global_fusion/dataset/gps_solution_TST2/hksc155d.20o")
nav_dat_n_d = gr.load("/persist/catkin_ws/src/GraphGNSSLib/global_fusion/dataset/gps_solution_TST2/hksc155d.20n")
nav_dat_b_d = gr.load("/persist/catkin_ws/src/GraphGNSSLib/global_fusion/dataset/gps_solution_TST2/hksc155d.20b")
nav_dat_l_d = gr.load("/persist/catkin_ws/src/GraphGNSSLib/global_fusion/dataset/gps_solution_TST2/hksc155d.20l")
nav_dat_g_d = gr.load("/persist/catkin_ws/src/GraphGNSSLib/global_fusion/dataset/gps_solution_TST2/hksc155d.20g")

In [3]:
print(nav_dat_l_c.attrs)

{'ionospheric_corr_GAL': [48.25, -0.011719, 0.0075684], 'version': 3.02, 'svtype': ['E'], 'rinextype': 'nav', 'filename': 'hksc155c.20l'}


In [4]:
obs_bag_msg_list = gen_bag_msg_list(obs_dat, '/gnss/meas')
print(obs_bag_msg_list[0][0], obs_bag_msg_list[-1][0])
base_timestamp_lambda = lambda x : x > obs_bag_msg_list[0][0] - 60 and x < obs_bag_msg_list[-1][0] + 60
# base_timestamp_lambda(1560089800)
base_bag_msg_list = gen_bag_msg_list(base_dat_d, '/gnss/base_c/meas', timestamp_lambda=base_timestamp_lambda) + gen_bag_msg_list(base_dat_d, '/gnss/base_d/meas', timestamp_lambda=base_timestamp_lambda)

1591153365.004 1591154350.005


In [5]:
ephem_timestamp_lambda = lambda x : x > obs_bag_msg_list[0][0] - 7300 and x < obs_bag_msg_list[-1][0] + 7300

In [6]:
b_nav_msg_list = gen_nav_bag_msg(nav_dat_b_c, timestamp_lambda=ephem_timestamp_lambda) + gen_nav_bag_msg(nav_dat_b_d, timestamp_lambda=ephem_timestamp_lambda)
n_nav_msg_list = gen_nav_bag_msg(nav_dat_n_c, timestamp_lambda=ephem_timestamp_lambda) + gen_nav_bag_msg(nav_dat_n_d, timestamp_lambda=ephem_timestamp_lambda)
l_nav_msg_list = gen_nav_bag_msg(nav_dat_l_c, timestamp_lambda=ephem_timestamp_lambda) + gen_nav_bag_msg(nav_dat_l_d, timestamp_lambda=ephem_timestamp_lambda)
g_nav_msg_list = gen_nav_bag_msg(nav_dat_g_c, timestamp_lambda=ephem_timestamp_lambda) + gen_nav_bag_msg(nav_dat_g_d, timestamp_lambda=ephem_timestamp_lambda)

G01 2108 266384.0 259110.0 GPSTime(week=2108, tow=266384.0) 2108
G11 2108 266384.0 259110.0 GPSTime(week=2108, tow=266384.0) 2108
G16 2108 266384.0 259110.0 GPSTime(week=2108, tow=266384.0) 2108
G03 2108 266400.0 259140.0 GPSTime(week=2108, tow=266400.0) 2108
G04 2108 266400.0 259140.0 GPSTime(week=2108, tow=266400.0) 2108
G07 2108 266400.0 259140.0 GPSTime(week=2108, tow=266400.0) 2108
G08 2108 266400.0 259140.0 GPSTime(week=2108, tow=266400.0) 2108
G09 2108 266400.0 259140.0 GPSTime(week=2108, tow=266400.0) 2108
G14 2108 266400.0 259140.0 GPSTime(week=2108, tow=266400.0) 2108
G21 2108 266400.0 259140.0 GPSTime(week=2108, tow=266400.0) 2108
G22 2108 266400.0 259140.0 GPSTime(week=2108, tow=266400.0) 2108
G26 2108 266400.0 259140.0 GPSTime(week=2108, tow=266400.0) 2108
G27 2108 266400.0 259140.0 GPSTime(week=2108, tow=266400.0) 2108
G30 2108 266400.0 259140.0 GPSTime(week=2108, tow=266400.0) 2108
G31 2108 266400.0 259140.0 GPSTime(week=2108, tow=266400.0) 2108
G04 2108 273584.0 266310.

In [7]:
bag_msg_list = obs_bag_msg_list + base_bag_msg_list
bag_msg_list.sort(key=lambda x: x[0])
print("{} -> {}".format(bag_msg_list[0][0], bag_msg_list[-1][0]))

1591153306.0 -> 1591154410.0


In [14]:
all_nav_msg_list_tmp = b_nav_msg_list + n_nav_msg_list + l_nav_msg_list + g_nav_msg_list
all_nav_msg_list = []
for timestamp, bag_msg, topic_name in all_nav_msg_list_tmp:
    # new_timestamp = timestamp - 7200
    # if new_timestamp < bag_msg_list[0][0] - 5:
    #     new_timestamp = bag_msg_list[0][0] - 5
    # if new_timestamp > bag_msg_list[-1][0] + 5:
    #     continue
    all_nav_msg_list.append((bag_msg_list[0][0] - 5, bag_msg, topic_name))
all_nav_msg_list_tmp = []

In [15]:
import pandas as pd
import numpy as np
import pymap3d as pm
gt_data = pd.read_csv("/persist/catkin_ws/src/GraphGNSSLib/global_fusion/dataset/gps_solution_TST2/groundTruth_TST.csv", sep=",", header=None, names=["timestamp", "lat", "lon", "alt"])


In [16]:
gt_ecef = gt_data.apply(lambda x: pm.geodetic2ecef(x["lat"], x["lon"], x["alt"]), axis=1)
gt_data["ecef_x"] = gt_ecef.apply(lambda x: x[0])
gt_data["ecef_y"] = gt_ecef.apply(lambda x: x[1])
gt_data["ecef_z"] = gt_ecef.apply(lambda x: x[2])
gt_data.to_csv("/persist/catkin_ws/src/GraphGNSSLib/global_fusion/dataset/gps_solution_TST2/groundTruth_TST_ecef.csv", index=False)

In [17]:
from sensor_msgs.msg import NavSatFix
import rospy
gt_msgs = []
for i in range(gt_data.shape[0]):
    gt_msg = NavSatFix()
    gt_msg.header.stamp = rospy.Time.from_sec(gt_data.iloc[i]["timestamp"])
    gt_msg.header.frame_id = "ground_truth"
    gt_msg.header.seq = i
    gt_msg.latitude = gt_data.iloc[i]["lat"]
    gt_msg.longitude = gt_data.iloc[i]["lon"]
    gt_msg.altitude = gt_data.iloc[i]["alt"]
    gt_msgs.append((gt_data.iloc[i]["timestamp"], gt_msg, '/groundtruth'))

In [18]:
all_bag_msg_list = bag_msg_list + all_nav_msg_list + gt_msgs
all_bag_msg_list.sort(key=lambda x: x[0])

In [19]:
import rosbag
import rospy
import time
import subprocess
# try:
#     roscore = subprocess.Popen('roscore')
#     time.sleep(1)  # wait a bit to be sure the roscore is really launched
# except:
#     print("roscore is already running")
bag = rosbag.Bag('test.bag', 'w')
iono_msg = StampedFloat64Array()
iono_msg.data = nav_dat_n_c.attrs['ionospheric_corr_GPS']
iono_msg.header.stamp = rospy.Time.from_sec(all_bag_msg_list[0][0])
bag.write('/gnss/c/iono_gps', iono_msg, t=rospy.Time.from_sec(all_bag_msg_list[0][0]))
iono_msg.data = nav_dat_b_c.attrs['ionospheric_corr_BDS']
bag.write('/gnss/c/iono_bds', iono_msg, t=rospy.Time.from_sec(all_bag_msg_list[0][0]))
iono_msg.data = nav_dat_l_c.attrs['ionospheric_corr_GAL']
bag.write('/gnss/c/iono_gal', iono_msg, t=rospy.Time.from_sec(all_bag_msg_list[0][0]))
iono_msg.data = nav_dat_n_d.attrs['ionospheric_corr_GPS']
bag.write('/gnss/d/iono_gps', iono_msg, t=rospy.Time.from_sec(all_bag_msg_list[0][0]))
iono_msg.data = nav_dat_b_d.attrs['ionospheric_corr_BDS']
bag.write('/gnss/d/iono_bds', iono_msg, t=rospy.Time.from_sec(all_bag_msg_list[0][0]))
iono_msg.data = nav_dat_l_d.attrs['ionospheric_corr_GAL']
bag.write('/gnss/d/iono_gal', iono_msg, t=rospy.Time.from_sec(all_bag_msg_list[0][0]))
for timestamp, bag_msg, topic_name in all_bag_msg_list:
    # print(timestamp, bag_msg, topic_name)
    bag.write(topic_name, bag_msg, t=rospy.Time.from_sec(timestamp))
bag.close()